Note: In this file, I have added the aliases from OpenCorporates after splitting the large file into batches.

In [1]:
import pandas as pd
import os
import dotenv
import requests
import json
from tqdm import tqdm

# set up environment variables
dotenv.load_dotenv()
OPENCORPORATES_API_KEY = os.getenv("OPENCORPORATES_API_KEY")

In [2]:
df = pd.read_csv('../data/openrefine-unique-notifying-parties.csv')

In [ ]:
df.head(20)

In [3]:
# group by 'Matched Name'
df_grouped = df.groupby('OpenCorporates URL').size()
# sort by size
df_grouped.sort_values(ascending=False)

OpenCorporates URL
http://opencorporates.com/companies/us_de/3736888        4
http://opencorporates.com/companies/lu/B153466           4
http://opencorporates.com/companies/fr/352045454         3
http://opencorporates.com/companies/bm/26108             3
http://opencorporates.com/companies/gb/02312079          3
                                                        ..
http://opencorporates.com/companies/de/M1201_HRB42023    1
http://opencorporates.com/companies/de/M1201_HRB41496    1
http://opencorporates.com/companies/de/M1201_HRB40601    1
http://opencorporates.com/companies/de/M1201_HRB39682    1
http://opencorporates.com/companies/za/2012-045177-07    1
Length: 6986, dtype: int64

### Retrieving previous company names

In [4]:
# Set up the OpenCorporates API URL
api_base_url = "https://api.opencorporates.com/v0.4/companies/{}/{}?api_token={}"

# Define a function to retrieve the previous names from the OpenCorporates API for a given URL
def get_previous_names(url):
    # Extract the jurisdiction code and company number from the OpenCorporates URL
    jurisdiction_code, company_number = url.split("/")[-2:]

    # Construct the OpenCorporates API URL using the jurisdiction code, company number, and your API key
    api_url = api_base_url.format(jurisdiction_code, company_number, OPENCORPORATES_API_KEY)

    # Make a GET request to the OpenCorporates API URL and parse the JSON response
    response = requests.get(api_url)
    data = json.loads(response.text)

    # Extract the previous names from the JSON response and return them
    return [name["company_name"] for name in data["results"]["company"].get("previous_names", [])]

# Add a new column 'previous_names' to the dataframe, containing the previous names for each row with an OpenCorporates URL
# df['previous_names'] = df.loc[df['OpenCorporates URL'].notnull(), 'OpenCorporates URL'].apply(get_previous_names)


In [9]:
# Split the original DataFrame into smaller batches with a maximum of 1,000 rows each
batch_size = 50
num_batches = df.shape[0] // batch_size + 1
batch_dfs = [df[i * batch_size:(i + 1) * batch_size] for i in range(num_batches)]

In [13]:
for i, batch_df in tqdm(enumerate(batch_dfs), total=num_batches, desc='Processing batches'):
    if i >= 187:  # Skip first 187 batches
        # Execute the desired line of code on the batch DataFrame
        batch_df['previous_names'] = batch_df.loc[batch_df['OpenCorporates URL'].notnull(), 'OpenCorporates URL'].apply(get_previous_names)
        
        # Save the modified batch DataFrame to a new CSV file
        batch_df.to_csv(f'batch_{i + 1}.csv', index=False)

Processing batches:   0%|          | 0/290 [00:00<?, ?it/s]/var/folders/0n/w0w76l850xd5bkqpcpl6v0rr0000gn/T/ipykernel_4735/3343747234.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['previous_names'] = batch_df.loc[batch_df['OpenCorporates URL'].notnull(), 'OpenCorporates URL'].apply(get_previous_names)
Processing batches:  65%|██████▍   | 188/290 [00:40<00:22,  4.60it/s]/var/folders/0n/w0w76l850xd5bkqpcpl6v0rr0000gn/T/ipykernel_4735/3343747234.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
# Combine all the smaller batch CSVs into a larger one
combined_df = pd.concat([pd.read_csv(f'batch_{i + 1}.csv') for i in range(num_batches)])

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('combined.csv', index=False)

In [16]:
# # Iterate over each row in the dataframe
# for index, row in df.iterrows():
#     # Check if the row has an OpenCorporates URL and previous names
#     if pd.notnull(row['OpenCorporates URL']) and row['previous_names']:
#         # Iterate over each previous name
#         for prev_name in row['previous_names']:
#             # Check if the previous name appears in the notifying_party column
#             if prev_name in df.loc[df['notifying_party'].notnull(), 'notifying_party'].values:
#                 # Set the OpenCorporates URL column of the current row to the URL of the row where the notifying_party matches the previous name
#                 df.loc[df['notifying_party'] == prev_name, 'OpenCorporates URL'] = row['OpenCorporates URL']
#                 # df.loc[index, 'OpenCorporates URL'] = df.loc[df['notifying_party'] == prev_name, 'OpenCorporates URL'].values[0]

# Use boolean indexing to filter rows that meet the conditions
mask = df['OpenCorporates URL'].notnull() & df['previous_names'].notnull()
filtered_df = df[mask]

# Extract unique previous names
unique_prev_names = set(prev_name for prev_names in filtered_df['previous_names'] for prev_name in prev_names)

# Use boolean indexing to filter rows where notifying_party matches previous name
mask = df['notifying_party'].isin(unique_prev_names)
df.loc[mask, 'OpenCorporates URL'] = filtered_df['OpenCorporates URL'].values[0]

In [ ]:
# save the dataframe as a CSV with the name 'openrefine-with-previous-names.csv'
df.to_csv('../data/openrefine-with-previous-names.csv', index=False)